# 2. Introduction à Pandas

<img align="center" src="https://habrastorage.org/files/10c/15f/f3d/10c15ff3dcb14abdbabdac53fed6d825.jpg"/>
<br>

**[Pandas](http://pandas.pydata.org)** est une librairire Python permettant l'analyse de données, elle est fréquemment utilisée par les data scientist pour lire et manipuler des données dans des formats tabulaires tel que `.csv`, `.tsv`, or `.xlsx`. Avec l'aide de `Matplotlib` et `Seaborn`, Pandas fournit des méthodes d'epxloration visuelles des données tabulaires.

## 2.1 DataFrame

Les structures de données principale de Pandas sont les **Series** et les **DataFrame**. La première est un tableau à une dimension contenant des données d'un type précis. La deuxième est une structure à deux dimensions, i.e. : un tableau à deux dimensions (ou encore une matrice), dont chaque colonne contient des données d'un type précis. Dans une `DataFrame` les lignes correspondent à des individus (objets, observations ...) et les colonnes à des attributs (features).

Nous commencons par les imports nécessaire pour que Pandas soit disponible dans notre environnement d'exécution :

In [ ]:
import numpy as np
import pandas as pd
# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')

Puis nous créons une DataFrame depuis un dictionnaire :

In [ ]:
cars = {'make': ['Ford', 'Honda', 'Toyota', 'Tesla'],
       'model': ['Taurus', 'Accord', 'Camry', 'Model S'],
       'MSRP': [27595, 23570, 23495, 68000]}          
df = pd.DataFrame(cars)   # creating DataFrame from dictionary
df                     # display the table

In [ ]:
print(df.index)       # print the row indices
print(df.columns)     # print the column indices

In [ ]:
df['year'] = 2018    # add column with same value
# add a column with a different value for each row:
df['dealership'] = ['Courtesy Ford','Capital Honda','Spartan Toyota',None]
df

Les éléments d'une DataFrame peuvent être accéder de différentes manières :

In [ ]:
# accessing an entire column will return a Series object

print(type(df['make']))
print(df['make'])    # returns the make column

In [ ]:
# accessing an entire row will return a Series object

print(type(df.iloc[2]))
print(df.iloc[2])       # returns the 3rd row of DataFrame

In [ ]:
# accessing a specific element of the DataFrame

print(df.iloc[1,2])    # retrieving second row, third column
print(df.loc[1,'MSRP'])    # retrieving second row, column named 'MSRP'
print(df.loc[1]['MSRP'])    # retrieving second row, column named 'MSRP'

In [ ]:
# accessing a slice of the DataFrame

df.iloc[1:3,1:3]

Les attributs shape et size permettent de connaitre le nombre de lignes et colonnes ainsi que le nombre d'élements :

In [ ]:
print('df.shape =', df.shape)    # shape returns a tuple (number of rows, number of columns)
print('df.size =', df.size)

## 2.2 Filtrage d'une DataFrame

La sélection d'une ou plusieurs lignes peut se faire en appliquant un filtre booléen :

In [ ]:
df[df.MSRP > 25000]

Le filtre est simplement constitué d'une Series contenant une valeur booléenne pour chaque ligne de notre DataFrame qui sera ensuite appliqué comme masque de sélection sur la DataFrame:

In [ ]:
df.MSRP > 25000

Il existe plusieurs méthodes utiles permettant de filtrer les DataFrames :

In [ ]:
df[df.dealership.isna()]    # retrieving rows with a null value on the dealership column

In [ ]:
df[df.MSRP.between(23000, 24000)]    # retrieving rows with MSRP between 23k and 24k

Les conditions dans un filtre peuvent être combinées avec les opérateurs `&` (ET booléen) et `|` (ou booléen) :

In [ ]:
# retrieving rows with MSRP < 30k and the model name contains an o
df[
    (df['MSRP'] < 30000) & df.model.str.contains('o')
]

## 2.3 Opérations arithmétiques

Pour illustrer ces opérations nous allons créer une DataFrame contenant des données numériques synthétiques à l'aide de numpy (numerical python):

In [ ]:
npdata = np.random.randn(5,3)  # create a 5 by 3 random matrix
columnNames = ['x1','x2','x3']
data = pd.DataFrame(npdata, columns=columnNames)
data

In [ ]:
print(data)

print('Data transpose operation:')
print(data.T)    # transpose operation

print('Addition:')
print(data + 4)    # addition operation

print('Multiplication:')
print(data * 10)   # multiplication operation

In [ ]:
print('data =')
print(data)

columnNames = ['x1','x2','x3']
data2 = pd.DataFrame(np.random.randn(5,3), columns=columnNames)
print('\ndata2 =')
print(data2)

print('\ndata + data2 = ')
print(data.add(data2))

print('\ndata * data2 = ')
print(data.mul(data2))

In [ ]:
print(data.abs())    # get the absolute value for each element

print('\nMaximum value per column:')
print(data.max())    # get maximum value for each column

print('\nMinimum value per row:')
print(data.min(axis=1))    # get minimum value for each row

print('\nSum of values per column:')
print(data.sum())    # get sum of values for each column

print('\nAverage value per row:')
print(data.mean(axis=1))    # get average value for each row

print('\nCalculate max - min per column')
f = lambda x: x.max() - x.min()
print(data.apply(f))

print('\nCalculate max - min per row')
f = lambda x: x.max() - x.min()
print(data.apply(f, axis=1))

In [ ]:
# add a max minus min column
data['max_minus_min'] = data.apply(
    lambda x: x.max() - x.min(),
    axis=1
)
data

## 2.4 Exercice

Nous reprennons le fichier `CO2_Emissions_Canada.csv`. Pour lire un fichier CSV nous pouvons utiliser la méthode `read_csv` de Pandas (la méthode `head()` affiche les 5 première lignes de notre DataFrame).

La description accompagnant ce dataset est la suivante :

**Model**
* 4WD/4X4 = Four-wheel drive
* AWD = All-wheel drive
* FFV = Flexible-fuel vehicle
* SWB = Short wheelbase
* LWB = Long wheelbase
* EWB = Extended wheelbase

**Transmission**
* A = Automatic
* AM = Automated manual
* AS = Automatic with select shift
* AV = Continuously variable
* M = Manual
* 3 - 10 = Number of gears

**Fuel type**
* X = Regular gasoline
* Z = Premium gasoline
* D = Diesel
* E = Ethanol (E85)
* N = Natural gas

**Fuel Consumption**

City and highway fuel consumption ratings are shown in litres per 100 kilometres (L/100 km) - the combined rating (55% city, 45% hwy) is shown in L/100 km and in miles per gallon (mpg)

**CO2 Emissions**

The tailpipe emissions of carbon dioxide (in grams per kilometre) for combined city and highway driving

Chargez le dataset contenu dans le fichier CO2 Emissions_Canada.csv dans une DataFrame avec la fonction [`pandas.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) et affichez ses premières lignes :

Affichez les dimensions de ce dataset, le nom des features et leurs types à l'aide des attributs `shape`, `columns` et de la méthode `info()` :

Utilisez la méthode `describe` pour afficher un ensemble de statistiques pour chaque feature numériques (`int64` et `float64`) : nombre de valeurs non manquantes, moyenne, variance, médiane, min, max, quartiles.

Répondez aux questions suivantes :
* Combien de ligne le fichier contient-il ?
* Quel est le CO2 maximum émis par km par un véhicule ?
* Combien de véhicules émettent une tel quantité de CO2 par km ?
* Combien de CO2 les véhicules émettent en moyenne ?
* Quels sont les 5 véhicules qui émettent le moins de CO2 par km (vous pouvez utiliser la méthode [`sort_values`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html)) ?
* Y-a-t'il des doublons dans ce dataset et si oui, combien (vous pouvez utiliser la méthode [`duplicated`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html)) ?
* Quelles sont les fabricants qui produisent les véhicules les plus polluants en moyenne (vous pouvez utiliser la méthode [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) - voir aussi [l'introduction tutorial de Pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/06_calculate_statistics.html#aggregating-statistics-grouped-by-category)) ?

## 2.7 Visualisation

Nous pouvons visualiser les données en utilisant directement les méthodes disponibles dans Pandas (qui utilisent la librairie matplotlib) :

In [ ]:
df['CO2 Emissions(g/km)'].hist(bins=100)    # distribution of CO2 emissions 

In [ ]:
df['Make'].value_counts().hist()    # number of vehicules per make

Ou la librairire Plotly (il est peut-être nécessaire de l'installer avant) :

In [ ]:
!pip  install plotly

In [ ]:
import plotly.express as px
fig = px.histogram(df, x="CO2 Emissions(g/km)")
fig.show()

In [ ]:
fig = px.histogram(df, x="Make").update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
fig = px.box(df, x="Make", y="CO2 Emissions(g/km)")
fig.show()

## 2.8 Excercice

* Afficher un box plot des émissions de CO2 par km par type de véhicule
* Afficher un [scatter plot](https://plotly.com/python/line-and-scatter/) des émissions de CO2 par km en fonction de la consommation de carburant
* Quel est le facteur qui explique les différentes modalités de cette relation ?